Import the data generated in BucklingBeams_createDataset.ipynb, and turn the graphs into fully connected graphs.

In [1]:
import pickle
import os

import numpy as np
import torch

In [ ]:
data_folder = r'path/to/data/folder'  # TODO: set this to your local data folder

In [3]:
data_path = os.path.join(data_folder, 'BucklingBeams_data_processed.pkl')
with open(data_path, 'rb') as f:
    data = pickle.load(f)

for graph in data['data_tr'][:5]:
    print(graph)

Data(edge_index=[2, 14], edge_attr=[14, 2], pos=[8, 2, 200, 2], node_attr=[8, 4], d=[1, 200], N=[1], lamb=[1, 200])
Data(edge_index=[2, 4], edge_attr=[4, 2], pos=[3, 2, 200, 2], node_attr=[3, 4], d=[1, 200], N=[1], lamb=[1, 200])
Data(edge_index=[2, 18], edge_attr=[18, 2], pos=[10, 2, 200, 2], node_attr=[10, 4], d=[1, 200], N=[1], lamb=[1, 200])
Data(edge_index=[2, 8], edge_attr=[8, 2], pos=[5, 2, 200, 2], node_attr=[5, 4], d=[1, 200], N=[1], lamb=[1, 200])
Data(edge_index=[2, 18], edge_attr=[18, 2], pos=[10, 2, 200, 2], node_attr=[10, 4], d=[1, 200], N=[1], lamb=[1, 200])


In [4]:
for key in ['data_tr', 'data_te']:
    for graph in data[key]:
        N = graph.N[0].item() # number of beam elements
        E_old = graph.edge_index.shape[1] # original number of edges

        # create new edges
        arr1, arr2 = np.meshgrid(np.arange(N+1), np.arange(N+1))
        new_edges = np.stack((arr1.flatten(), arr2.flatten()), axis=-1).T
        new_edges = torch.tensor(new_edges[:, np.abs(new_edges[0] - new_edges[1]) > 1])
        graph.edge_index = torch.cat((graph.edge_index, new_edges), dim=1)

        # calculate length of new edges
        pos0 = graph.pos[:,:,0,0] # positions at time step 0, 1st solution
        L_i_new = torch.norm(pos0[new_edges[0]] - pos0[new_edges[1]], dim=1, keepdim=True)

        # stiffness of new edges is zero
        C_i_new = torch.zeros_like(L_i_new)

        # update edge attributes
        edge_attr2 = torch.cat((L_i_new, C_i_new), dim=1)
        graph.edge_attr = torch.cat((graph.edge_attr, edge_attr2), dim=0)

        # add edge attribute that indicates virtual vs real edges
        new_attr = -torch.ones(N*(N+1), 1,
                              dtype=graph.edge_attr.dtype, device=graph.edge_attr.device)
        new_attr[:2*N] = -1
        graph.edge_attr = torch.cat((graph.edge_attr, new_attr), dim=1)

        # break


In [5]:
with open('../data/BucklingBeams_data_fullyConnected.pkl', 'wb') as f:
    pickle.dump(data, f)